這份 Notebook 示範 OpenAI Function Calling 基本用法


In [4]:
# from google.colab import userdata
# openai_api_key = userdata.get('openai_api_key')

In [5]:
# Import necessary libraries
## 設定 OpenAI API Key 變數
from dotenv import load_dotenv
import os

# Load the environment variables from .env file
load_dotenv()

# Access the API key
openai_api_key = os.getenv('OPENAI_API_KEY')


In [6]:
import requests
import json
from pprint import pp

In [7]:
def get_completion(messages, model="gpt-3.5-turbo", temperature=0, max_tokens=1000, tools=None, tool_choice=None):
  payload = { "model": model, "temperature": temperature, "messages": messages, "max_tokens": max_tokens }
  if tools:
    payload["tools"] = tools
  if tool_choice:
    payload["tool_choice"] = tool_choice

  headers = { "Authorization": f'Bearer {openai_api_key}', "Content-Type": "application/json" }
  response = requests.post('https://api.openai.com/v1/chat/completions', headers = headers, data = json.dumps(payload) )
  obj = json.loads(response.text)

  if response.status_code == 200 :
    return obj["choices"][0]["message"] # 改成回傳上一層 message 物件
  else :
    return obj["error"]

## 基本的完整流程

出自官方案例 https://openai.com/blog/function-calling-and-other-api-updates

## Step 1

發出 Prompt 請求，附上你有的 function 規格

In [8]:
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)


In [9]:
messages = [{"role": "user", "content": "今天台北市的天氣如何?"}]
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    }
]

response = get_completion(messages, tools=tools)
pp(response)

{'role': 'assistant',
 'content': None,
 'tool_calls': [{'id': 'call_K4qiWN1FiQBZYUYjjFVEdMOx',
                 'type': 'function',
                 'function': {'name': 'get_current_weather',
                              'arguments': '{"location":"Taipei, Taiwan"}'}}],
 'refusal': None,
 'annotations': []}


## Step 2

你實際呼叫 get_current_weather 方法，帶有參數 location，得到一個結果

In [10]:
# 把 AI 的回覆加到對話歷史紀錄
messages.append(response)

# 函數名稱與函數物件的對應
available_functions = {
  "get_current_weather": get_current_weather,
}

# response 若有 function_call，表示 GPT 要我呼叫函數
if response.get("tool_calls"):
  for tool_call in response.get("tool_calls"):
    # GPT 要我執行哪一個函數
    function_name = tool_call["function"]["name"]

    # 函數物件
    fuction_to_call = available_functions[function_name]

    # 擷取出函式的參數
    function_args = json.loads(tool_call["function"]["arguments"])

    # 實際呼叫函數
    function_response = fuction_to_call(
      function_args.get("location"),
      unit=function_args.get("unit"),
    )

    # 把函數的回傳結果，塞回對話紀錄，角色是 tool
    messages.append(
        {
            "tool_call_id": tool_call["id"],
            "role": "tool",
            "name": function_name,
            "content": function_response,
        }
    )

    pp(function_response)

('{"location": "Taipei, Taiwan", "temperature": "72", "unit": null, '
 '"forecast": ["sunny", "windy"]}')


## Step 3

 再次呼叫 OpenAI API，此時 messages 裡面有三個訊息: user, assistant 和 function 結果:

In [11]:
pp(messages)

[{'role': 'user', 'content': '今天台北市的天氣如何?'},
 {'role': 'assistant',
  'content': None,
  'tool_calls': [{'id': 'call_K4qiWN1FiQBZYUYjjFVEdMOx',
                  'type': 'function',
                  'function': {'name': 'get_current_weather',
                               'arguments': '{"location":"Taipei, Taiwan"}'}}],
  'refusal': None,
  'annotations': []},
 {'tool_call_id': 'call_K4qiWN1FiQBZYUYjjFVEdMOx',
  'role': 'tool',
  'name': 'get_current_weather',
  'content': '{"location": "Taipei, Taiwan", "temperature": "72", "unit": '
             'null, "forecast": ["sunny", "windy"]}'}]


In [12]:
response = get_completion(messages)
pp(response)

{'role': 'assistant',
 'content': '今天台北市的天氣為晴天，有微風，氣溫為攝氏 72 度。祝您有個愉快的一天！',
 'refusal': None,
 'annotations': []}


## 換一個問題試看看

如果問題沒有需要呼叫 function 呢?

In [13]:
messages = [{"role": "user", "content": "寫一首關於天氣的詩"}]

response = get_completion(messages, tools=tools)
pp(response)

{'role': 'assistant',
 'content': '在蔚藍的天空下\n'
            '陽光灑在肩頭\n'
            '微風輕拂臉龐\n'
            '溫暖如春的懷抱\n'
            '\n'
            '雲朵悠悠飄過\n'
            '像是天空的畫布\n'
            '彩虹穿越雨霧\n'
            '帶來希望的訊息\n'
            '\n'
            '雨滴輕輕落下\n'
            '清澈如心靈的歌\n'
            '大地滿溢生機\n'
            '讓我們感受自然的美好\n'
            '\n'
            '無論晴天或雨天\n'
            '天氣總是變幻莫測\n'
            '但在每一刻裡\n'
            '都能感受到生命的溫暖與美麗',
 'refusal': None,
 'annotations': []}


回傳的 response 裡面就沒有 tool_calls

## Fake function 用法: 擷取 metadata

透過一個 fake function，目的是拿結構化的 function 參數

In [14]:
messages = [
  {"role": "user", "content": "台積電 2023/5/1 的法說會資料 blah blah blah blah"}
]

tools = [
    {
        "type": "function",
        "function": {
            "name": "information_extraction",
            "description": "Extracts the relevant information from the passage.",
            "parameters": {
                "type": "object",
                "properties": {
                    "company_name": {
                        "type": "string",
                        "description": "報告中的公司名稱",
                    },
                    "report_date": {
                        "type": "string",
                        "description": "報告中的日期"
                    },
                }
            },
        },
    }
]

# 這個 tool_choice 參數可以要求 GPT 一定要執行某個函數，預設是 auto 讓 GPT 自行判斷
tool_choice =  {"type": "function", "function": {"name": "information_extraction"}}

response = get_completion(messages, tools=tools, tool_choice=tool_choice)
pp(response)

{'role': 'assistant',
 'content': None,
 'tool_calls': [{'id': 'call_4vmB62Oh3ztdHuNgKUN4Nojp',
                 'type': 'function',
                 'function': {'name': 'information_extraction',
                              'arguments': '{"company_name":"台積電","report_date":"2023/5/1"}'}}],
 'refusal': None,
 'annotations': []}


In [15]:
metadata = json.loads(response["tool_calls"][0]["function"]["arguments"])
pp(metadata)

{'company_name': '台積電', 'report_date': '2023/5/1'}


不過在 JSON mode 這功能出了之後，就算不用 function calling 我們也可以拿到 JSON 格式

所以似乎不一定要用 function calling 這招來擷取 metadata 了，看哪一種可以更節省 tokens 數

### 使用 Google Search 工具

In [16]:
# 這是非官方的 google 爬蟲
!pip install googlesearch-python

# 若要用官方 JSON API https://developers.google.com/custom-search/v1/overview?hl=zh-tw (有 API key 需付費但有免費額度)

Defaulting to user installation because normal site-packages is not writeable


In [17]:
from googlesearch import search

In [18]:
def google_search(keyword):
  content = ""
  for item in search(keyword, advanced=True, num_results=5, lang='zh-tw'):
    content += f"Title: {item.title}\n Description: {item.description}\n\n"
  return content

In [19]:
messages = [{"role": "user", "content": "今天台北天氣如何?"}]

tools = [
    {
        "type": "function",
        "function": {
            "name": "google_search",
            "description": "搜尋最新的資訊",
            "parameters": {
                "type": "object",
                "properties": {
                    "keyword": {
                        "type": "string",
                        "description": "搜尋關鍵字",
                    }
                },
                "required": ["keyword"],
            },
        },
    }
]

response = get_completion(messages, tools=tools)
pp(response)

{'role': 'assistant',
 'content': None,
 'tool_calls': [{'id': 'call_0oySOCYWLSH5pM1uxa9m1Ry7',
                 'type': 'function',
                 'function': {'name': 'google_search',
                              'arguments': '{"keyword":"台北天氣"}'}}],
 'refusal': None,
 'annotations': []}


In [20]:
metadata = json.loads(response["tool_calls"][0]["function"]["arguments"])
pp(metadata)

{'keyword': '台北天氣'}


In [21]:
messages.append(response)
available_functions = {
  "google_search": google_search,
}


if response.get("tool_calls"):
  for tool_call in response.get("tool_calls"):
    function_name = tool_call["function"]["name"]
    fuction_to_call = available_functions[function_name]
    function_args = json.loads(tool_call["function"]["arguments"])
    function_response = fuction_to_call(**function_args)

    messages.append(
        {
            "tool_call_id": tool_call["id"],
            "role": "tool",
            "name": function_name,
            "content": function_response,
        }
    )

    pp(function_response)

('Title: \n'
 ' Description: /\n'
 '\n'
 'Title: 臺北市- 縣市預報| 交通部中央氣象署\n'
 ' Description:  今日白天 陰陣雨或雷雨 26 - 2979 - 84降雨機率90%舒適至悶熱; 今晚明晨 多雲時陰陣雨或雷雨 26 - '
 '2879 - 82降雨機率70%舒適至悶熱; 明日白天 陰時多雲短暫陣雨或雷雨\xa0... \n'
 '\n'
 'Title: 臺北市, 台北市, 臺灣三日天氣預報 - AccuWeather\n'
 ' Description:  空氣品質對大多數人來說很理想；享受您的正常戶外活動吧。 過敏展望. 檢視全部 · 灰塵與微粒 極高 此位置\xa0'
 '... \n'
 '\n'
 'Title: 臺北市信義區- 鄉鎮預報| 交通部中央氣象署\n'
 ' Description:  臺北市信義區 · 72小時預報 一週溫度曲線 一週體感溫度曲線 過去 24小時 · 72小時預報 一週預報 過去24小時 '
 '· 臺北市信義區即時影像 · 天氣圖資 · 臺北市附近旅遊景點天氣連結. \n'
 '\n'
 'Title: 東榮, 臺北市10 天天氣預報 - The Weather Channel\n'
 ' Description:  10 日天氣-東榮, 臺北市. 截至08:15 CST 為止. 今日. 30°/26°. 99%. 白天. 30°. '
 '99%. 西北西17 km/h. 雨。 高溫30°C。 10 到15 公里/小時的西北西風。 \n'
 '\n')


In [22]:
response = get_completion(messages)
pp(response)

{'role': 'assistant',
 'content': '根據中央氣象局的資料，今天台北白天陰天，有陣雨或雷雨，氣溫介於26至29度之間，降雨機率高達90%。今晚至明晨多雲時陰，仍有陣雨或雷雨，氣溫約26至28度，降雨機率為70%。明天白天則是陰時多雲，可能會有短暫陣雨或雷雨。整體來說，天氣悶熱舒適，請注意保暖和攜帶雨具。',
 'refusal': None,
 'annotations': []}
